# WHEN OBJECTS ARE ALIKE

In the programming world, duplicate code is considered evil.
 
We should not have multiple copies of the same, or similar, code in different places. 

When we fix a bug in one copy and fail to fix the same bug in another copy, we've caused no end of problems for ourselves.



## INHERITANCE

Technically every class we create uses inheritance.

All Python classes are subclasses of the special class named `object`.

If we don't explicitly inherit from a different class, our classes will automatically inherit from object.

All classes automatically inherit from object if we don't explicitly provide a different superclass (parent class).

 Inheritance requires a minimal amount of extra syntax over a basic class definition. 
 
Simply include the name of the parent class inside parentheses between the class name and the colon that follows.
 
This is all we have to do to tell Python that the new class should be derived from the given superclass.

### How To Apply Inheritance

The simplest and most obvious use of inheritance is to add functionality to an existing class.

Let's start with a contact manager that tracks names and email addresses.

The `contact` class is responsible for maintaining global list of all contacts ever seen in a class variable, and for initializing the name and address for an individual contact:

In [1]:
class Contact:
    
    all_contacts: list["Contact"] = []
    
    def __init__(self, name: str = "" , email: str= "" ) -> None:
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)
        
    def __repr__(self) -> str:
        return (
            f"{self.__class__.__name__}("
            f"{self.name!r}, {self.email!r}"
            f")"
      )
        
        
    

This example introduces us to class variables.

The `all_contacts` list,because it is part of the class definition, is shared by all instances of this class.

This means that there is only one `Contact.all_contacts` list.

We can also access it as `self.all_contacts` from within any method on an instance of the `Contact` class.

If a field cannot be found on the object via `self`, then it will be found on the class and will thus refer to the same single list.




#### PRO-TIP

Be careful with the `self`-based reference.

It can only provide access to an existing class-based variable.

If you evert attempt to set the variable using `self.all_contacts`, you will actually be creating a new instance variable associated just with that object.

The class variable will still be unchanged and accesible as `Contact.all_contacts`

In [2]:
c_1 = Contact("Sevval", "sevvalunver@gmail.com")
c_2 = Contact("Serdar", "serdarunver@gmail.com")

In [3]:
Contact.all_contacts

[Contact('Sevval', 'sevvalunver@gmail.com'),
 Contact('Serdar', 'serdarunver@gmail.com')]

This is a simple class that allows us to track a couple of pieces of data about each contact. 

But what if some of our contacts are also suppliers that we need to order supplies from? 

We could add an `order` method to the Contact class, but that would allow people to accidentally order things from contacts who are customers or family friends. 

Instead, let's create a new `Supplier` class that acts like our `Contact` class, but has an additional `order` method that accepts a yet-to-be-defined `Order` object:

In [4]:
class Supplier(Contact):
    
    def order(self, order: "Order") -> None:
        print(
            "If this were a real system we would send "
            f"'{order}' order to '{self.name}'"
        )

If we test this class in our trusty interpreter, we see that all contacts, including suppliers, accept a name and email address in their ``__init__`` method, but that only `suppliers` instance have an `order` method:

In [5]:
c = Contact("Some Body", "somebody@somename.com")
s = Supplier("Sup Plier", "supplier@example.com")

In [6]:
print(c.name, c.email, s.name, s.email)

Some Body somebody@somename.com Sup Plier supplier@example.com


In [7]:
from pprint import pprint
pprint(c.all_contacts)

[Contact('Sevval', 'sevvalunver@gmail.com'),
 Contact('Serdar', 'serdarunver@gmail.com'),
 Contact('Some Body', 'somebody@somename.com'),
 Supplier('Sup Plier', 'supplier@example.com')]


In [8]:
c.order("I need pliers")

AttributeError: 'Contact' object has no attribute 'order'

In [ ]:
s.order("I need pliers")

Supplier class can do everything a Contact can do (including adding itself to the list of Contact.all_contacts), but it also has an additional method that allows us to place orders.

If we used `self.all_contacts` then this would not collect all objects into the `Contact` class, but would put `Supplier` instances into `Supplier.all_contacts`. 

## Extending Built-Ins

One interesting use of inheritance is adding functionality to built-in types (classes).

In the `Contact` class seen earlier, we are adding contacts to a list of all contacts.

What if we wanted to search that list by name?

We could add a metgod on the `Contact` class to do that, but it feels like this method actually belongs to the list itself.

We need to import `annotations` module from `__future__` package.


In [ ]:
from __future__ import annotations

class ContactList(list["Contact"]):
    
    def search(self, name: str) -> list["Contact"]:
        """Return all contacts that contain the search value in their name."""
        matching_contacts: list["Contact"] = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts
    
class Contact:
    
        all_contacts: ContactList = ContactList()
        
        def __init__(self, name: str = "", email: str = "") -> None:
            self.name = name
            self.email = email
            Contact.all_contacts.append(self)
            
        def __repr__(self) -> str:
            return (
                f"{self.__class__.__name__}("
                f"{self.name!r}, {self.email!r}"
                f")"
            )

In [ ]:
c1 = Contact("John A", "johna@example.com")
c2 = Contact("John B", "johnb@example.com")
c3 = Contact("Jenna C", "cutty@example.com")

In [ ]:
[c.name for c in Contact.all_contacts.search("John")]

Instead of instatiating a gneric list as our class variable, we create a new `ContactList` class that extends the built-in `list` class.

As a second example, we can extend the `dict` class, which is a collection of keys and their associated values.

We can create instances of dictionaries using `{}` syntax.

An extended class that tracts the longest ket it has seen:

In [ ]:
class LongNameDict(dict[str, int]):
    
    def longest_key(self) -> Optional[str]:
        longest = None
        for key in self:
            if longest is None or len(key) > len(longest):
                longest = key
        return longest

The hint for the class narrowed the generic dict to a more specific dict[str, int]; the keys are of type str and the values are of type int.
 
This helps mypy reason about the longest_key() method. Since the keys are supposed to be str-type objects, the statement for key in self: will iterate over str objects. 

The result will be a str, or possibly None. That's why the result is described as `Optional[str]`.

We're going to be working with strings and integer values. Perhaps the strings are usernames, and the integer values are the number of articles they've read on a website.
 
In addition to the core username and reading history, we also need to know the longest name so we can format a table of scores with the right size display box.

In [ ]:
articles_read = LongNameDict()

articles_read["Doug"] = 34
articles_read["Anne"] = 11
articles_read["Boris"] = 42

In [ ]:
articles_read.longest_key()

Most built-in types can be similarly extended. These built-in types fall into several interesting families, with separate kinds of type hints:

### 1-) Generic Collections:

`set`,`list`,`dict`.

These use type hints like `set[something]`, `list[something]`, and `dict[key,value]` to narrow the hint from purely generic to something more specific that the application will actually use.

To use the generic types as annotations, a `from __future__ import annotations` is required.

### 2-)

The `typing.NamedTuple` definition lets us define new kinds of immutable tuples and provide useful names for the members.

### 3-)

Python has type hints for file-related I/O objects. A new kind of file can use a type hint of `typing.TextIO` or `typing.BinaryIO` to describe built-file file operations.

### 4-)

It's possible to create new types of strings by extending `typing.Text`. For the most part, the built-in `str` class does everything we need.

### 5-)

New numeric types often start with the `numbers` module as a source for built-in numeric functionality.

## Overriding and Super

Our `Contact` class allows only a name and an email address. This may be sufficient for most contacts, but what if we want to add a phone number for our close friends?

We could do this easily by setting a `phone` attribute on the contact after it is constructed.

But if we want to make this third variable available on initialization, we have to override the `__init__` method from the superclass.

Overriding means altering or replacing a method of the superclass with a new method in the subclass.

No special syntax is needed to do this; the subclass's newly created method is automatically called instead of the superclass's method.

In [ ]:
class Friend(Contact):
    
    def __init__(self, name: str = "", email: str = "", phone: str = "") -> None:
        self.name = name
        self.email = email
        self.phone = phone

Our Contact and Friend classes have duplicate code to set up the name and email properties; this can make code maintenance complicated, as we have to update the code in two or more places. 

More alarmingly, our Friend class is neglecting to add itself to the all_contacts list we have created on the Contact class. 

Finally, looking forward, if we add a feature to the Contact class, we'd like it to also be part of the Friend class.

What we really need is a way to execute the original __init__() method on the Contact class from inside our new class. 

This is what the super() function does; it returns the object as if it was actually an instance of the parent class, allowing us to call the parent method directly:


In [ ]:
class Friend(Contact):
    
    def __init__(self, name: str = "", email: str = "", phone: str = "") -> None:
        super().__init__(name, email)
        self.phone = phone

This example first binds the instance to the parent class using `super()` and calls `__init__()` on that object, passing in the expected arguments.

It then does its own initialization, namely, setting the `phone` attribute, which is unique to the `Friend` class.

The `Contact` class provided a definition for the `__repr__()` method to produce a string representation of the object.

Our class did not override the `__repr__()` method inherited from the superclass.

In [ ]:
f = Friend("Sevvalism26", "ss@example", "12345")

In [ ]:
Contact.all_contacts

The details shown for a Friend instance don't include the new attribute. 

It's easy to overlook the special method definitions when thinking about class design.

A super() call can be made inside any method. Therefore, all methods can be modified via overriding and calls to super(). 

The call to super() can also be made at any point in the method; we don't have to make the call as the first line. 

For example, we may need to manipulate or validate incoming parameters before forwarding them to the superclass. 

# Multiple Inheritance

Multiple inheritance in principle is simple: a subclass that inherits from more than one parent class can access functionality from both of them.

In practice, it requires some care to be sure any method overrides are fully understood.

The simplest and most useful form of multiple inheritance follows a design pattern called the **mixin**.

A mixin class definition is not intended to exist on its own; but is meant to be inherited by some other class to provide extra functionality.

For example, let's say we wanted to add functionality to our `Contact` class that allows sending an email to `self.email`.

Sending mail is a common task that we might want to use on many other classes. So, we can write a simple mixin class that provides this functionality:


In [ ]:
from typing import Protocol


class Emailable(Protocol):
    email: str
    
class MailSender(Emailable):
    
    def send_mail(self, message: str) -> None:
        print(f"Sending mail to {self.email}")
        # Add e-mail logic here

The `MailSender` class does not do anything special.

We have two classes because we are describing two things: aspects of the host class for a mixin, and new aspects the mixin provides to the host.

We needed to create a hint, `Emailable`, to describe the kinds of classes or `MailSender` mixin expects to work with.

This kind of type hint is called a `Protocol`.

`Protocols` generally have methods and can also have a class-level attribute names with type hints but not full assignment statements.

A protocol tells that any class (or Subclass) of `Emailable` object must support an `email` attribute of type `str`.

Note that we are relying on Python's name resolution rules.

The name `self.email` can be resolved as either an instance variable, or a class-level variable, `Emailable.email`, or a property.

The mypy tool will check all the classes mixed in with `MailSender` for instance- or class-level definitions. 

We only need to provide the name of the attribute at the class level, with a type hint to make it clear to mypy that the mixin does not define the attribute – the class into which it's mixed will provide the `email` attribute.

Because of Python's duck typing rules, we can use the `MailSender` mixin with any class that has an `email` attribute defined. 

A class with which `MailSender` is mixed doesn't have to be a formal subclass of `Emailable`; it only has to provide the required attribute.

For brevity, we didn't include the actual email logic here; if you're interested in studying how it's done, see the `smtplib` module in the Python standard library.

The `MailSender` class does allow us to define a new class that describes both a `Contact` and a `MailSender`, using multiple inheritance:

In [ ]:
class EmailableContact(Contact, MailSender):
    pass


The syntax for multiple inheritance looks like a parameter list in the class definition. 

Instead of including one base class inside the parentheses, we include two (or more), separated by a comma. 

When it's done well, it's common for the resulting class to have no unique features of its own. 

It's a combination of mixins, and the body of the class definition is often nothing more than the pass placeholder.

We can test this new hybrid to see the mixin at work:

In [ ]:
e = EmailableContact("John Smith", "jognb@sloop.net")

In [ ]:
Contact.all_contacts

In [ ]:
e.send_mail("Hello, test e-mail here")

The `Contact` initializer is still adding the new contact to the `all_contacts` list, and the mixin is able to send mail to `self.email`, so we know that everything is working.

let's consider some other options we had for this example, rather than using a mixin:

We could have used single inheritance and added the `send_mail` function to a subclass of `Contact`. 

The disadvantage here is that the email functionality then has to be duplicated for any unrelated classes that need an email. 

For example, if we had email information in the payments part of our application, unrelated to these contacts, and we wanted a `send_mail()` method, we'd have to duplicate the code.

We can create a standalone Python function for sending an email, and just call that function with the correct email address supplied as a parameter when the email needs to be sent (this is a very common choice). 

Because the function is not part of a class, it's harder to be sure that proper encapsulation is being used.

We could explore a few ways of using composition instead of inheritance. 

For example, `EmailableContact` could have a `MailSender` object as a property instead of inheriting from it. 

This leads to a more complex `MailSender` class because it now has to stand alone. It also leads to a more complex `EmailableContact` class because it has to associate a `MailSender` instance with each `Contact`.


We could try to monkey patch (we'll briefly cover monkey patching in Chapter 13, Testing Object-Oriented Programs) the `Contact` class to have a `send_mail` method after the class has been created. 

This is done by defining a function that accepts the `self` argument, and setting it as an attribute on an existing class. 

This is fine for creating a unit test fixture, but terrible for the application itself.


Multiple inheritance works alright when we are mixing methods from different classes, but it can be problamatic when we have to call methods on the superclass.

When there are multiple superclasses, how do we know which one's methods to call?

What is the rule for selecting the appropriate superclass method?

Let's explore these questions by adding address to Friend Class:

Approaches that we can take are:

1-) 
An address is a collection of strings representing the street, city, country, and other related details of the contact. 

We could pass each of these strings as a parameter into the `Friend` class's `__init__()` method. 

We could also store these strings in a generic tuple or dictionary. These options work well when the address information doesn't need new methods.

2-) 
Another option would be to create our own `Address` class to hold those strings together, and then pass an instance of this class into the `__init__()` method in our Friend class. 

The advantage of this solution is that we can add behavior (say, a method to give directions or to print a map) to the data instead of just storing it statically. This is an example of composition.
 
The "has-a" relationship of composition is a perfectly viable solution to this problem and allows us to reuse Address classes in other entities, such as buildings, businesses, or organizations. 

3-)

A third course of action is a cooperative multiple inheritance design. While this can be made to work, it doesn't pass muster with mypy. 

The reason, we'll see, is some potential ambiguity that's difficult to describe with the available type hints.

----------------------------------------------------------------------------------------------------------------------------





The objective here is to add a new class to hold an address.

We will call this new class `AddresHolder` instead of Address because inheritance defiens an " is-a" relationship, and an address is not a contact.

In [ ]:
class AddressHolder:
    
    def __init__(self, street: str, city: str, state: str, code: str) -> None:
        self.street = street
        self.city = city
        self.state = state
        self.code = code

## The Diamond Problem

We can use multiple inheritance to add thiss class as a parent of our existing `Friend` class.

The tricky part is that we now have two parent `__init__()` methods, both of which needed to be called.

And they need to be called wtih different arguments.

We could start with a naive approach for the `Friend` class:

In [ ]:
class Friend(Contact, AddressHolder):
    
    def __init__(
        self,
        name: str = "",
        email: str = "",
        phone: str = "",
        street: str = "",
        city: str = "",
        state: str = "",
        code: str = "",
    ) -> None:
        Contact.__init__(self, name, email)
        AddressHolder.__init__(self, street, city, state, code)
        self.phone = phone

In above example, we directly call the `__init__()` function on each of the superclasses and explicitly pass the `self` argument.

This example technically works, we can access the different variables directly on the class.

But there are few problems with this approach.

First, it is possible for a superclass to remain uninitialized if we neglect to explicitly call the initializer.

That would not be a problem for this example but it could cause hard to debug program crashes in common scenarios.



More insidious possiblity is a superclass being called multiple times because of the organization of the class hierarchy.

![UML1](./uml1.png)

The `__init__()` method from the `Friend` class calls the `__init__()` method on the `Contact` class, which implicitly initializes the `object` superclass.

The `Friend` class then calls `__init__()` on the `AddressHolder` , which implicitly initializes the object superclass again.

This means the parent class has been set up twice.

With the `object` class, that's relatively harmless, but in some sitations, it could bring disaster.

Base class should onlşy be called once. Once , yes, but when?

Do we call `Friend`, then `Contact`, then `object`, then `AddressHolder`?

Or `Friend` then `Contact`, then `AddressHolder`, then `object`?

Let's contrive an example to illustrate the problem:

![UML2](./uml2.png)

We have a base class `BaseClass` that has a method named `call_me()`.

Two subclasses, `LeftSubclass` and `RightSubclass`, inherit from `BaseClass`.

Finally, we have two subclasses of both `LeftSubclass` and `RightSubclass` named `Subclass`\ extends the `BaseClass`\ and each overrides the `call_me()` method with different implementation.

Then, another subclass extends both of these using multiple inheritance with a fourth, distinct implementation of the `call_me()` method. 

This is called **diamond inheritance** because of the diamond shape of the class diagram.

In [ ]:
class BaseClass:
    num_base_calls = 0
    def call_me(self) -> None:
        print("Calling method on BaseClass")
        self.num_base_calls += 1
class LeftSubclass(BaseClass):
    num_left_calls = 0
    def call_me(self) -> None:
        BaseClass.call_me(self)
        print("Calling method on LeftSubclass")
        self.num_left_calls += 1
class RightSubclass(BaseClass):
    num_right_calls = 0
    def call_me(self) -> None:
        BaseClass.call_me(self)
        print("Calling method on RightSubclass")
        self.num_right_calls += 1
class Subclass(LeftSubclass, RightSubclass):
    num_sub_calls = 0
    def call_me(self) -> None:
        LeftSubclass.call_me(self)
        RightSubclass.call_me(self)
        print("Calling method on Subclass")
        self.num_sub_calls += 1


This example ensures that each overriden `call_me()` method directly calls the parent method wtih the same name.

It let's us know each time a method is called by printing the information to the screen.

It also creates a distinct instance variable to show how many times it has been called.

If we instantiate one Subclass object and call the call_me() method on it once, we get the following output:

In [ ]:
s = Subclass()
s.call_me()

In [ ]:
print(s.num_sub_calls,
      s.num_left_calls,
      s.num_right_calls,
      s.num_base_calls)

Thus, we can see the base class's `call_me()` method being called twice. 

This could lead to some pernicious bugs if that method is doing actual work, such as depositing into a bank account, twice.

Python's Method Resolution Order (MRO) algorithm transforms the diamond into a flat, linear tuple.

We can see the results of this in the `__mro__` attribute of a class. 

The linear version of this diamond is the sequence Subclass, LeftSubclass, RightSubClass, BaseClass, object. 

What's important here is that Subclass lists LeftSubclass before RightSubClass, imposing an ordering on the classes in the diamond.

The thing to keep in mind with multiple inheritance is that we often want to call the next method in the MRO sequence, not necessarily a method of the parent class. 

The `super()` function locates the name in the MRO sequence. Indeed, `super()` was originally developed to make complicated forms of multiple inheritance possible.

Here is the same code written using `super()`.
 
We've renamed some of the classes, adding an _S to make it clear this is the version using `super()`: 

In [ ]:
class BaseClass:
    num_base_calls = 0
    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1
class LeftSubclass_S(BaseClass):
    num_left_calls = 0
    def call_me(self) -> None:
        super().call_me()
        print("Calling method on LeftSubclass_S")
        self.num_left_calls += 1
class RightSubclass_S(BaseClass):
    num_right_calls = 0
    def call_me(self) -> None:
        super().call_me()
        print("Calling method on RightSubclass_S")
        self.num_right_calls += 1
class Subclass_S(LeftSubclass_S, RightSubclass_S):
    num_sub_calls = 0
    def call_me(self) -> None:
        super().call_me()
        print("Calling method on Subclass_S")
        self.num_sub_calls += 1


The change is pretty minor; we only replaced the naive direct calls with calls to super(). 

The Subclass_S class, at the bottom of the diamond, only calls super() once rather than having to make the calls for both the left and right. 

The change is easy enough, but look at the difference when we execute it:

In [ ]:
ss = Subclass_S()
ss.call_me()

In [ ]:
print(
ss.num_sub_calls,
ss.num_left_calls,
ss.num_right_calls,
ss.num_base_calls
)


This output looks good: our base method is only being called once. We can see how this works by looking at the `__mro__` attribute of the class:

In [ ]:
from pprint import pprint

pprint(Subclass_S.__mro__)

The order of the classes shows what order `super()` will use. 

The last class in the tuple is generally the built-in `object` class. 

As noted earlier in this chapter, it's the implicit superclass of all classes.

This shows what `super()` is actually doing. 

Since the print statements are executed after the `super` calls, the printed output is in the order each method is actually executed. 

Let's look at the output from back to front to see who is calling what:

------------------------------------

We start with the Subclass_S.call_me() method. This evaluates super().call_me(). The MRO shows LeftSubclass_S as next.

We begin evaluation of the LeftSubclass_S.call_me() method. This evaluates super().call_me(). The MRO puts RightSubclass_S as next. This is not a superclass; it's adjacent in the class diamond.

The evaluation of the RightSubclass_S.call_me() method, super().call_me(). This leads to BaseClass.

TheBaseClass.call_me() method finishes its processing: printing a message and setting an instance variable, self.num_base_calls, to BaseClass.num_base_calls + 1.

Then, the RightSubclass_S.call_me() method can finish, printing a message and setting an instance variable, self.num_right_calls.

Then, the LeftSubclass_S.call_me() method will finish by printing a message and setting an instance variable, self.num_left_calls.

This serves to set the stage for Subclass_S to finish its call_me() method processing. It writes a message, sets an instance variable, and rests, happy and successful.

--------------

Pay particular attention to this: 

The super call is not calling the method on the superclass of LeftSubclass_S (which is BaseClass). 

Rather, it is calling RightSubclass_S, even though it is not a direct parent of LeftSubclass_S! 

This is the next class in the MRO, not the parent method. 

RightSubclass_S then calls BaseClass and the super() calls have ensured each method in the class hierarchy is executed once.

## DIFFERENT SETS OF ARGUMENTS

This is going to make things complicated as we return to our `Friend` class example.

In the `__init__()` method of the `Friend` class, we were originally delegating initialization to the `__init__()` methods of both parent classes, wtih different sets of arguments.

In [ ]:
Contact.__init__(self, name, email)

In [ ]:
AddressHolder.__init__(self, street, city, state, code)

How can we manage different sets of arguments when using `super()`? 

We only really have access to the next class in the MRO sequence. 

Because of this, we need a way to pass the extra arguments through the constructors so that subsequent calls to `super()`, from other mixin classes, receive the right arguments.

It works like this. The first call to `super()` provides arguments to the first class of the MRO, passing the name and email arguments to `Contact.__init__()`. 

Then, when `Contact.__init__()` calls `super()`, it needs to be able to pass the address-related arguments to the method of the next class in the MRO, which is `AddressHolder.__init__()`.

This problem often manifests itself anytime we want to call superclass methods with the same name, but with different sets of arguments. 

Collisions often arise around the special method names. Of these, the most common example is having a different set of arguments to various `__init__()` methods, as we're doing here.

There's no magical Python feature to handle cooperation among classes with divergent `__init__()` parameters. 

Consequently, this requires some care to design our class parameter lists. 

The cooperative multiple inheritance approach is to accept keyword arguments for any parameters that are not required by every subclass implementation. 

A method must pass the unexpected arguments on to its `super()` call, in case they are necessary to later methods in the MRO sequence of classes.

While this works and works well, it's difficult to describe with type hints. 

Python's function parameter syntax provides a tool we can use to do this, but it makes the overall code look cumbersome. 

Have a look at a version of the Friend multiple inheritance code:

In [ ]:
class Contact:
    all_contacts = ContactList()
    def __init__(self, /, name: str = "", email: str = "", **kwargs: Any) -> None:
        super().__init__(**kwargs)  # type: ignore [call-arg]
        self.name = name
        self.email = email
        self.all_contacts.append(self)
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(" f"{self.name!r}, {self.email!r}" f")"
class AddressHolder:
    def __init__(
        self,
        /,
        street: str = "",
        city: str = "",
        state: str = "",
        code: str = "",
        **kwargs: Any,
    ) -> None:
        super().__init__(**kwargs)  # type: ignore [call-arg]
        self.street = street
        self.city = city
        self.state = state
        self.code = code
class Friend(Contact, AddressHolder):
    def __init__(self, /, phone: str = "", **kwargs: Any) -> None:
        super().__init__(**kwargs)
        self.phone = phone


We've added the `**kwargs` parameter, which collects all additional keyword argument values into a dictionary. 

When called with `Contact(name="this", email="that", street="something")`, the `street` argument is put into the `kwargs` dictionary; these extra parameters are passed up to the next class with the `super()` call.
 
The special parameter / separates parameters that could be provided by position in the call from parameters that require a keyword to associate them with an argument value. 

We've given all string parameters an empty string as a default value, also.

If you aren't familiar with the `**kwargs` syntax, it basically collects any keyword arguments passed into the method that were not explicitly listed in the parameter list. 

These arguments are stored in a dictionary named `kwargs` (we can call the variable whatever we like, but convention suggests kw or kwargs). 

When we call a method, for example, `super().__init__()`, with `**kwargs` as an argument value, it unpacks the dictionary and passes the results to the method as keyword arguments. 

Multiple inheritance following the mixin pattern often works out very nicely. 

The idea is to have additional methods defined in mixin classes, but to keep all of the attributes centralized in a host class hierarchy. 

This can avoid the complexity of cooperative initialization.

Design using composition also often works better than complex multiple inheritance. 

The inheritance paradigm depends on a clear "is-a" relationship between classes. 

Multiple inheritance folds in other relationships that aren't as clear. 

We can say that an "Email is a kind of Contact," for example. But it doesn't seem as clear that we can say "A Customer is an Email." 

We might say "A Customer has an Email address" or "A Customer is contacted via Email," using "has an" or "is contacted by" instead of a direct "is-a" relationship.

# Polymorphism

It is a showy name describing a simple concept: 

different behaviors happen depending on which subclass is being used, without having to explicitly know what the subclass actually is.

We should be able to substitute any subclass for its superclass.

As an example, imagine a program that plays audio files. 

A media player might need to load an AudioFile object and then play it. 

We can put a play() method on the object, which is responsible for decompressing or extracting the audio and routing it to the sound card and speakers. 

The act of playing an AudioFile could feasibly be as simple as:

However, the process of decompressing and extracting an audio file is very different for different types of files. 

While .wav files are stored uncompressed, .mp3, .wma, and .ogg files all utilize totally different compression algorithms.

We can use inheritance with polymorphism to simplify the design. 

Each type of file can be represented by a different subclass of AudioFile, for example, WavFile and MP3File. 

Each of these would have a play() method that would be implemented differently for each file to ensure that the correct extraction procedure is followed. 

The media player object would never need to know which subclass of AudioFile it is referring to; it just calls play() and polymorphically lets the object take care of the actual details of playing.

Polymorphism is one of the most important reasons to use inheritance in many object-oriented contexts. 

Because any objects that supply the correct interface can be used interchangeably in Python, it reduces the need for polymorphic common superclasses. 

Inheritance can still be useful for sharing code, but if all that is being shared is the public interface, duck typing is all that is required.

This reduced need for inheritance also reduces the need for multiple inheritance; often, when multiple inheritance appears to be a valid solution, we can just use duck typing to mimic one of the multiple superclasses.